In [1]:
#!pip install requests beautifulsoup4
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
id_estaciones = {
    "centro de alto rendimiento": "S_5",
    "guaymaral": "S_8",
    "las ferias": "S_6",
    "minambiente": "S_17",
    "suba": "S_11",
    "usaquen": "S_1",
    "ciudad bolivar": "S_37",
    "colina": "S_38",
    "jazmin": "S_34",
    "movil fontibon": "S_39",
    "usme": "S_32",
    "san cristobal": "S_24",
    "tunal": "S_4",
    "bolivia": "S_27",
    "carvajal - sevillana": "S_3",
    "fontibon": "S_30",
    "kennedy": "S_9",
    "movil 7ma": "S_26",
    "puente aranda": "S_13"
}


In [6]:
def extraer_datos(nombre, fecha):
    # Construir la URL para hacer la solicitud
    url = f"http://rmcab.ambientebogota.gov.co/Report/HourlyReports?id=1&UserDateString={fecha}"
    response = requests.get(url)
        
    id = id_estaciones[nombre] # Obtener el ID de la estación a partir del nombre

    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        # Parsear el contenido HTML de la respuesta
        soup = BeautifulSoup(response.text, 'html.parser')

        # Buscar el contenedor de la tabla
        contenedor_tabla = soup.select_one(f'div#{id}.TableStation[data-val="sda"][data-station="{nombre}"]')

        if contenedor_tabla:
            # Parsear la tabla dentro del contenedor
            rows = BeautifulSoup(str(contenedor_tabla), 'html.parser').find_all('tr')
            # Extraer los encabezados de la tabla
            headers = [th.text.strip() for th in rows[0].find_all('th')]
            # Extraer los datos de cada fila y almacenarlos en una lista
            data = [[td.text.strip() for td in row.find_all('td')] for row in rows[1:]]
            # Crear un DataFrame con los datos extraídos
            df = pd.DataFrame(data, columns=headers)
        else:
            print("No se encontró una tabla dentro del contenedor.")
    else:
        print(f"Falla al obtener la página. Código de estado: {response.status_code}")
        return None  # Detener ejecución si la página no se carga correctamente

    # Concatenar los textos de las columnas 'parámetros' y 'Unidades'
    df['Hora'] = df['parámetros'] + ' [' + df['Unidades'] + ']'

    # Tomar solo las columnas con los datos de interés y establecer 'Hora' como índice
    df.drop(columns=['parámetros', 'Unidades', 'Máx', 'Mín'], inplace=True)
    df.set_index('Hora', inplace=True)    
    datos_interes = ['PM10 [µg/m3]', 'PM2.5 [µg/m3]']#, 'Vel Viento [m/s]', 'Dir Viento [Grados]']
    df = df.loc[datos_interes]

    # Organizar la forma del dataframe y agregar la fecha al indice
    df = df.transpose()
    df.drop('', axis=0, inplace=True)
    df.index = fecha + ' ' + df.index + ':00'

    return df

In [4]:
#Para Lun 22 de Enero de 2024 a Mie 31 de Enero de 2024
# Iterar sobre cada estación en el diccionario id_estaciones
for estacion in id_estaciones.keys(): 
    dia_inicial = 23
    dia_final = 31
    # Extraer los datos del primer día y almacenarlos en un DataFrame
    df = extraer_datos(estacion, f"2024-01-{dia_inicial}") 

    # Iterar sobre los días del rango para extraer y concatenar datos adicionales
    for i in range(dia_inicial, dia_final + 1):
        fecha = f"2024-01-{i}"       
        df_temp = extraer_datos(estacion, fecha)  # Extraer los datos para el día actual        
        df = pd.concat([df, df_temp]) # Concatenar los datos extraídos en un solo DataFrame 

    # Guardar el DataFrame por estación en un archivo CSV
    df.to_csv(f"../Data/{estacion}_2024-01-{dia_inicial}_{dia_final}.csv")

In [9]:
#Para Lun 23 de Enero de 2023 a Mie 01 de Febrero de 2023
# Iterar sobre cada estación en el diccionario id_estaciones
for estacion in id_estaciones.keys(): 
    dia_inicial = 23
    dia_final = 31
    # Extraer los datos del primer día y almacenarlos en un DataFrame
    df = extraer_datos(estacion, f"2023-01-{dia_inicial}") 

    # Iterar sobre los días del rango para extraer y concatenar datos adicionales
    for i in range(dia_inicial, dia_final + 1):
        fecha = f"2023-01-{i}"       
        df_temp = extraer_datos(estacion, fecha)  # Extraer los datos para el día actual        
        df = pd.concat([df, df_temp]) # Concatenar los datos extraídos en un solo DataFrame 
    
    fecha = f"2023-02-01"       
    df_temp = extraer_datos(estacion, fecha)       
    df = pd.concat([df, df_temp]) 


    # Guardar el DataFrame por estación en un archivo CSV
    df.to_csv(f"../Data/{estacion}_2023-01-{dia_inicial}_2023-02-01.csv")